In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics # for the evaluation
from sklearn.preprocessing import LabelEncoder,MinMaxScaler
import tensorflow as tf 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM


In [2]:
df = pd.read_csv("data/mongo_data.csv", index_col='ts')
df = df.drop('Unnamed: 0', 1)
#Remove cols with the same value

def unique_cols(df):
    a = df.to_numpy() # df.values (pandas<0.24)
    return (a[0] == a).all(0)

df = df.loc[:,np.invert(unique_cols(df))]
print(df.shape)
df.tail()

C:\Users\pedro\AppData\Local\Temp\ipykernel_49016\468073868.py:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  df = df.drop('Unnamed: 0', 1)


(467334, 39)


,S_SUM,U_L1_L2,I_SUM,F,U_L3_L1,C_phi_L1,U_L3_N,RealEc_L1,RealEc_L2,RealEc_L3,...,S_L3,Q_L1,S_L2,U_L2_N,S_L1,U_L1_N,I_L1,RealEc_SUM,I_L2,I_L3
ts,,,,,,,,,,,,,,,,,,,,,
2022-02-22T11:17:24.127703575Z,123259.65,411.39,1.12,50.01,411.12,0.99,236.57,31345348,30444914,30360338,...,42346.98,6085.66,39469.35,238.10,41443.33,237.23,174.70,92150704,165.77,179.00
2022-02-22T11:17:25.439348217Z,122868.31,411.33,1.05,50.01,409.30,0.98,236.51,31345362,30444928,30360352,...,40454.14,7272.30,39607.47,237.89,42806.71,236.40,181.08,92150752,166.49,171.04
2022-02-22T11:17:26.705083973Z,123108.94,411.02,1.04,50.01,410.57,0.98,237.13,31345374,30444940,30360364,...,40749.97,8029.02,41030.93,237.73,41328.03,237.20,174.23,92150784,172.60,171.85
2022-02-22T11:17:28.014494747Z,123048.30,410.90,1.02,50.01,410.75,0.99,236.20,31345388,30444952,30360380,...,42542.77,5632.11,39168.28,237.67,41337.26,237.06,174.37,92150824,164.80,180.11
2022-02-22T11:17:29.306443661Z,122208.24,409.81,1.08,50.01,409.76,0.99,235.70,31345400,30444964,30360392,...,41760.77,5949.70,39547.37,237.24,40900.10,236.52,172.93,92150864,166.70,177.18


In [8]:
from sklearn.decomposition import PCA
pca = PCA(n_components=10)

ds = pca.fit_transform(df)
print(pca.n_components_)

pcn = pca.n_components_

10


In [9]:
# get the index of the most important feature on EACH component
most_important = [np.abs(pca.components_[i]).argmax() for i in range(pcn)]
initial_feature_names = df.columns
# get the most important feature names
most_important_names = [initial_feature_names[most_important[i]] for i in range(pcn)]

In [12]:
df[most_important_names]

,AE_SUM,S_SUM,AE_L3,AE_L1,AE_L2,Q_SUM,Q_L2,P_L2,ReacE_L1,ReacE_L1
ts,,,,,,,,,,
2022-02-15T10:24:58.903000343Z,86568392,146233.91,28678250,29258780,28631288,21902.44,4561.12,44363.30,5341039.0,5341039.0
2022-02-15T10:24:59.374508444Z,86568448,145337.03,28678270,29258800,28631306,21888.11,7289.41,43622.85,5341041.5,5341041.5
2022-02-15T10:25:00.655871379Z,86568496,145837.16,28678286,29258816,28631324,21141.56,5764.87,44042.22,5341044.5,5341044.5
2022-02-15T10:25:01.887888454Z,86568552,146147.23,28678304,29258836,28631342,21412.03,6832.28,43739.84,5341047.0,5341047.0
2022-02-15T10:25:03.145591485Z,86568600,145390.61,28678320,29258852,28631358,21970.95,8086.44,44012.39,5341049.0,5341049.0
...,...,...,...,...,...,...,...,...,...,...
2022-02-22T11:17:24.127703575Z,104822744,123259.65,34766736,35423928,34632004,22909.30,7764.83,34431.22,6419695.5,6419695.5
2022-02-22T11:17:25.439348217Z,104822792,122868.31,34766752,35423944,34632020,22202.75,5588.35,35195.03,6419698.5,6419698.5
2022-02-22T11:17:26.705083973Z,104822832,123108.94,34766768,35423960,34632032,23349.25,7287.72,36270.11,6419700.5,6419700.5
